In [1]:
##
import os
import sys
import warnings
import enum
from enum import Enum
import matplotlib
from pathlib import Path
from typing import Callable

##
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

##
from finrl import config as finrl_config
from finrl.main import check_and_make_directories
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import (
    TEST_END_DATE,
    TRAINED_MODEL_DIR,
)
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.plot import backtest_stats, convert_daily_return_to_pyfolio_ts, get_baseline

##
import yfinance as yf

##
from meta.data_processors.yahoofinance import Yahoofinance

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/tensorboard/compat/proto/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/tensorboard/compat/proto/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors 

In [2]:
def config():
    # #
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # INFO, WARN are ignored and only ERROR messages will be printed

    # #
    matplotlib.use("Agg")

    # #
    check_and_make_directories(
        [
            finrl_config.DATA_SAVE_DIR,
            finrl_config.TRAINED_MODEL_DIR,
            finrl_config.TENSORBOARD_LOG_DIR,
            finrl_config.RESULTS_DIR,
        ]
    )


config()

In [5]:
##
import numpy as np
import pandas as pd
from pyfolio import timeseries
import tqdm

##
from rl.data import DataTechnicalAnalysis, DataFundamentalAnalysis, DataBase
from common.utils import now_time
from configuration.settings import ProjectDir
from DataBase import MissingDataHandler, get_missing_counts

In [6]:
_TRAIN_DATA_START = "2010-01-01"
_TRAIN_DATA_END = "2021-12-31"
_TEST_DATA_START = "2021-01-01"
_TEST_DATA_END = "2021-12-31"
tickers = DOW_30_TICKER

In [12]:
def print_info(df: pd.DataFrame):
    ##
    print(f"{df.shape=}")
    print(f"Start date: {df['date'].iloc[0]}")
    print(f"End date: {df['date'].iloc[-1]}")
    print(f"Stocks: {df['tic'].unique().size}")

    ##
    grouped_data_size = df.groupby(by=["date"]).size()
    if grouped_data_size.nunique() == 1:
        print(f"OK: All dates have same row count: {grouped_data_size.unique()}")
    else:
        print(f"ERROR: Different size for each date: {grouped_data_size.unique()}")

In [8]:
print(tickers.__len__())
print(tickers)
data_processor = DataBase(
    data_source="yahoofinance", start_date=_TRAIN_DATA_START, end_date=_TEST_DATA_END, time_interval="1d"
)

30
['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [9]:
data_processor.download_data(ticker_list=tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [10]:
print(data_processor.dataframe.shape)
data_processor.dataframe.head()
print_info(data_processor.dataframe)

(88283, 9)
df.shape=(88283, 9)
Start date: 2010-01-04
End date: 2021-12-30
Stocks: 30


ValueError: Different size for each date: [29 30]

In [11]:
mh = MissingDataHandler()
df_filled = mh.fill_missed_tic_gaps(data_processor.dataframe, tickers)

100%|██████████| 2317/2317 [00:51<00:00, 45.38it/s]


In [13]:
df_filled.shape

(90600, 9)

In [14]:
fe = FeatureEngineer(use_technical_indicator=True, use_turbulence=False, user_defined_feature=False)
data_features: pd.DataFrame = fe.preprocess_data(df_filled.copy())
data_features = data_features.sort_values(["date", "tic"]).reset_index(drop=True)

Successfully added technical indicators


In [15]:
data_features.shape

(90600, 17)

,date,open,high,low,close,adjusted_close,volume,tic,day
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.515212,493729600,AAPL,0
1,2010-01-04,56.630001,57.869999,56.560001,57.720001,43.267174,5277400,AMGN,0
2,2010-01-04,40.810001,41.099998,40.389999,40.919998,33.792694,6894300,AXP,0
3,2010-01-04,55.720001,56.389999,54.799999,56.180000,43.777546,6186700,BA,0
4,2010-01-04,57.650002,59.189999,57.509998,58.549999,41.353195,7325600,CAT,0
...,...,...,...,...,...,...,...,...,...
88278,2021-12-30,507.929993,509.230011,503.649994,504.429993,498.051575,1309800,UNH,3
88279,2021-12-30,217.970001,219.149994,217.149994,217.869995,216.234299,3812800,V,3
88280,2021-12-30,52.380001,52.570000,52.080002,52.250000,49.538273,15615500,VZ,3
88281,2021-12-30,52.360001,52.919998,51.939999,51.990002,49.691002,3653600,WBA,3


In [19]:
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))
filename = data_processor.get_filename(prj_dir=prj_dir, name="dji30_ta")
data_processor.save_dataset(data_features, filename)

OK: Dir exist for: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-15T16-21-06.json
Saving data into: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-15T16-21-06.json
Data saved to json: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-15T16-21-06.json


In [52]:
def add_covariance_as_nested_states(df: pd.DataFrame) -> pd.DataFrame:
    ##
    df = df.sort_values(["date", "tic"], ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []
    return_list = []

    # look back is one year
    lookback = 252
    for i in tqdm.tqdm(range(lookback, len(df.index.unique()))):
        data_lookback = df.loc[i - lookback : i, :]
        price_lookback = data_lookback.pivot_table(index="date", columns="tic", values="close")
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values

        #
        return_list.append(return_lookback)
        cov_list.append(covs)

    df_cov = pd.DataFrame({"date": df.date.unique()[lookback:], "cov_list": cov_list, "return_list": return_list})
    # df["covariance_matrix"] = df_cov
    # df = df.sort_values(["date", "tic"]).reset_index(drop=True)
    return df_cov


# TODO: fixme add covariance as states
df_cov = add_covariance_as_nested_states(data_features.copy())
# TODO: merge to dataset

  0%|          | 0/2768 [08:27<?, ?it/s]


KeyboardInterrupt: 

In [66]:
# print(df_cov.shape)
# print(df_cov['cov_list'].shape)
# print(df_cov['return_list'].shape)

# df_cov['cov_list'] = df_cov['cov_list'].apply(lambda x: pd.DataFrame(x))
# df_cov['cov_list']
# _d3 = pd.DataFrame(df_cov['cov_list'][0])
# _d4 = df_cov['cov_list']
# _d3 = pd.DataFrame(df_cov['return_list'][0])
# _d4 = df_cov['return_list'][0]
# df_cov['return_list']
# _d1 = df_cov['cov_list']
# _d2 = df_cov['return_list']

(2768, 3)
(2768,)
(2768,)


In [40]:
df_cov.to_json("foo.json")

In [26]:

df_cov1 = pd.DataFrame({"cov_list":})


0       [[0.0002849618759954093, 9.196035527740299e-05...
1       [[0.0002850066689925423, 9.224443498717904e-05...
2       [[0.0002838971487853813, 9.16941940901012e-05,...
3       [[0.0002838698218188037, 9.158961530444638e-05...
4       [[0.00028388999474777423, 9.158103382271378e-0...
                              ...                        
2763    [[0.00025416445352502334, 4.899979240800397e-0...
2764    [[0.00025586787418637315, 4.9525971145642e-05,...
2765    [[0.0002513291755320819, 4.902713173926194e-05...
2766    [[0.00025048848339192836, 4.9832109315140924e-...
2767    [[0.0002503516522709732, 5.0038111289378386e-0...
Name: cov_list, Length: 2768, dtype: object

In [37]:
print_info(data_processor.dataframe)

df.shape=(0, 0)


KeyError: 'date'

In [29]:
data_features_cov1.head()

,date,open,high,low,close,adjusted_close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2011-01-03,11.630000,11.795000,11.601429,11.770357,10.033262,445138400,AAPL,0,0.118248,11.709899,11.326780,62.862180,144.286559,25.488753,11.415357,11.212333,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
1,2011-01-03,55.200001,56.279999,55.180000,55.549999,41.640541,5453300,AMGN,0,0.261720,58.568809,52.495191,50.863117,34.401812,4.927279,54.949000,55.565166,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
2,2011-01-03,43.299999,43.619999,43.110001,43.400002,36.488426,7633300,AXP,0,-0.178628,47.098241,41.260759,51.443736,-26.423899,13.049417,43.883000,42.473000,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
3,2011-01-03,66.150002,66.680000,66.000000,66.400002,53.034523,8072900,BA,0,-0.081580,66.520526,63.197475,51.659856,149.651676,11.924376,64.853001,66.948501,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
4,2011-01-03,94.379997,94.809998,94.110001,94.150002,68.190575,5231500,CAT,0,2.266966,96.271975,88.886025,68.687627,82.330220,50.929600,90.157333,85.266833,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...


In [15]:
_d = data_features_cov1["cov_list"][0]
_d

NameError: name 'data_features_cov1' is not defined